In [774]:
import pandas as pd
import numpy  as np

df = pd.read_csv('googleplaystore.csv')

# check the fraction of missing values per row and column.
print(df.isnull().sum(axis=0)/df.shape[0])
print(sum(df.isnull().sum(axis=1)!=0)/df.shape[0])


App               0.000000
Category          0.000000
Rating            0.135965
Reviews           0.000000
Size              0.000000
Installs          0.000000
Type              0.000092
Price             0.000000
Content Rating    0.000092
Genres            0.000000
Last Updated      0.000000
Current Ver       0.000738
Android Ver       0.000277
dtype: float64
0.13661101374411955


In [775]:
# fill in Types since it can be implied from Price
a = df.Type.isna()
for i in range(len(df.Type)):
    if a[i] == True:
        if df.Price[i] == "0":
            df.Type[i] = "Free"
        else:
            df.Type[i] = "Paid"

df.Type.isna().any()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


False

In [776]:
# treat missing values in Content Rating
df["Content Rating"].replace(to_replace=' ',value = "?", inplace=True)
df.Type.isna().any()

False

In [777]:
# drop missing values
df.dropna(inplace = True)   #remove missing entries
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 10840
Data columns (total 13 columns):
App               9360 non-null object
Category          9360 non-null object
Rating            9360 non-null float64
Reviews           9360 non-null object
Size              9360 non-null object
Installs          9360 non-null object
Type              9360 non-null object
Price             9360 non-null object
Content Rating    9360 non-null object
Genres            9360 non-null object
Last Updated      9360 non-null object
Current Ver       9360 non-null object
Android Ver       9360 non-null object
dtypes: float64(1), object(12)
memory usage: 1023.8+ KB


In [778]:
# convert the type of Reviews from object to int
df['Reviews'] = df['Reviews'].astype(int)
df.Reviews.dtypes

dtype('int64')

In [779]:
df.Size

0                       19M
1                       14M
2                      8.7M
3                       25M
4                      2.8M
                ...        
10834                  2.6M
10836                   53M
10837                  3.6M
10839    Varies with device
10840                   19M
Name: Size, Length: 9360, dtype: object

In [780]:
#convert Size into a uniform standard of numeric
df.Size = df.Size.str.replace('k','e+3')
df.Size = df.Size.str.replace('M','e+6')

def is_convertable(v):
    try:
        float(v)
        return True
    except ValueError:
        return False
    
temp = df.Size.apply(lambda x: is_convertable(x))
temp.head()

df.Size[~temp].value_counts()

Varies with device    1637
Name: Size, dtype: int64

In [781]:
# deal with "Varies with device"(temporarily set to 0, but better add another column)
df.Size = df.Size.replace('Varies with device','0')

df.Size = pd.to_numeric(df.Size)
df.Size.isna().any()
df.Size

0        19000000.0
1        14000000.0
2         8700000.0
3        25000000.0
4         2800000.0
            ...    
10834     2600000.0
10836    53000000.0
10837     3600000.0
10839           0.0
10840    19000000.0
Name: Size, Length: 9360, dtype: float64

In [782]:
# seen as a categorical feature and find the unique values
a = df.Installs.unique()
a

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '100+', '500+', '10+',
       '5+', '50+', '1+'], dtype=object)

In [783]:
df.Price.unique()

array(['0', '$4.99', '$3.99', '$6.99', '$7.99', '$5.99', '$2.99', '$3.49',
       '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49', '$10.00',
       '$24.99', '$11.99', '$79.99', '$16.99', '$14.99', '$29.99',
       '$12.99', '$2.49', '$10.99', '$1.50', '$19.99', '$15.99', '$33.99',
       '$39.99', '$3.95', '$4.49', '$1.70', '$8.99', '$1.49', '$3.88',
       '$399.99', '$17.99', '$400.00', '$3.02', '$1.76', '$4.84', '$4.77',
       '$1.61', '$2.50', '$1.59', '$6.49', '$1.29', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$8.49', '$1.75', '$14.00', '$2.00',
       '$3.08', '$2.59', '$19.40', '$3.90', '$4.59', '$15.46', '$3.04',
       '$13.99', '$4.29', '$3.28', '$4.60', '$1.00', '$2.95', '$2.90',
       '$1.97', '$2.56', '$1.20'], dtype=object)

In [784]:
# remove the "$" and convert to numeric
df.Price = df.Price.apply(lambda x: x.strip('$'))
df.Price = pd.to_numeric(df.Price)
df.Price.unique()

array([  0.  ,   4.99,   3.99,   6.99,   7.99,   5.99,   2.99,   3.49,
         1.99,   9.99,   7.49,   0.99,   9.  ,   5.49,  10.  ,  24.99,
        11.99,  79.99,  16.99,  14.99,  29.99,  12.99,   2.49,  10.99,
         1.5 ,  19.99,  15.99,  33.99,  39.99,   3.95,   4.49,   1.7 ,
         8.99,   1.49,   3.88, 399.99,  17.99, 400.  ,   3.02,   1.76,
         4.84,   4.77,   1.61,   2.5 ,   1.59,   6.49,   1.29, 299.99,
       379.99,  37.99,  18.99, 389.99,   8.49,   1.75,  14.  ,   2.  ,
         3.08,   2.59,  19.4 ,   3.9 ,   4.59,  15.46,   3.04,  13.99,
         4.29,   3.28,   4.6 ,   1.  ,   2.95,   2.9 ,   1.97,   2.56,
         1.2 ])

In [785]:
df.Genres

0                     Art & Design
1        Art & Design;Pretend Play
2                     Art & Design
3                     Art & Design
4          Art & Design;Creativity
                   ...            
10834                    Education
10836                    Education
10837                    Education
10839            Books & Reference
10840                    Lifestyle
Name: Genres, Length: 9360, dtype: object

In [786]:
# split the genres
split_genres1 = df.Genres.apply(lambda x: x.split(';')[0])
split_genres2 = df.Genres.apply(lambda x: x.split(';')[-1])
df["First Genre"]=split_genres1
df["Second Genre"]=split_genres2

In [787]:
df["Last Updated"]

0         January 7, 2018
1        January 15, 2018
2          August 1, 2018
3            June 8, 2018
4           June 20, 2018
               ...       
10834       June 18, 2017
10836       July 25, 2017
10837        July 6, 2018
10839    January 19, 2015
10840       July 25, 2018
Name: Last Updated, Length: 9360, dtype: object

In [788]:
# convert it to the date using datetime package
from datetime import datetime,date
update_date=pd.to_datetime(df["Last Updated"])
update_date.head()

0   2018-01-07
1   2018-01-15
2   2018-08-01
3   2018-06-08
4   2018-06-20
Name: Last Updated, dtype: datetime64[ns]

In [789]:
# convert to the days till now to get more inspirations
df["Updated Days"] = update_date.apply(lambda x:date.today()-datetime.date(x))
df["Updated Days"]

0        632 days
1        624 days
2        426 days
3        480 days
4        468 days
           ...   
10834    835 days
10836    798 days
10837    452 days
10839   1716 days
10840    433 days
Name: Updated Days, Length: 9360, dtype: timedelta64[ns]

In [790]:
# convert to numeric
df["Updated Days"] = df["Updated Days"].astype(int)
df["Updated Days"] = df["Updated Days"] / 864e+11
df["Updated Days"]

0         632.0
1         624.0
2         426.0
3         480.0
4         468.0
          ...  
10834     835.0
10836     798.0
10837     452.0
10839    1716.0
10840     433.0
Name: Updated Days, Length: 9360, dtype: float64

In [791]:
df["Current Ver"]

0                     1.0.0
1                     2.0.0
2                     1.2.4
3        Varies with device
4                       1.1
                ...        
10834                 1.0.0
10836                  1.48
10837                   1.0
10839    Varies with device
10840    Varies with device
Name: Current Ver, Length: 9360, dtype: object

In [792]:
# change into the format of number.number
import re
df["Current Ver"]=df["Current Ver"].apply(lambda x: 'Varies with device' if x=='Varies with device'  else  re.findall('^[0-9]\.[0-9]|[\d]|\W*',str(x))[0])

In [793]:
df["Current Ver"].unique()

array(['1.0', '2.0', '1.2', 'Varies with device', '1.1', '6.1', '2.9',
       '2.8', '3.8', '3.1', '2.2', '5.5', '4.0', '1.5', '6.0', '6.7',
       '1.6', '2.1', '1.3', '1', '1.4', '3.0', '1.7', '2.5', '2.4', '1.9',
       '0.2', '4.4', '2.3', '8.3', '3.2', '4.6', '5.4', '3.3', '9.1',
       '5.0', '9.0', '7.0', '3.9', '3.4', '9.7', '4.9', '5.2', '3.5',
       '7.3', '5.1', '4.1', '9', '0.9', '6.4', '3.7', '2.6', '4.3', '6.3',
       '4.7', '4.2', '1.8', '3', '9.8', '2', '4', '', '6', '7.5', '9.9',
       '0.1', '7.7', '7.2', '6.5', '5.6', '4.5', '5.8', '8.1', '6.6',
       '4.8', '8.7', '0.0', '3.6', '5.3', '8.2', '5.7', '8.5', '0', '8.0',
       '8.4', '6.8', '6.2', '2.7', '5.9', '7.8', '7.1', '7.9', '8.8',
       '7.4', '9.6', '8', '9.4', '5', '0.7', '0.5', '7', '7.6', '0.6',
       '8.9', '6.9', '9.3', '9.2', '8.6', '9.5', '0.8', '0.4', '0.3'],
      dtype=object)

In [794]:
# deal with "Varies" and empty values
df["Current Ver"].replace(to_replace="Varies with device",value = "0", inplace=True)
df["Current Ver"].replace(to_replace="",value = "0", inplace=True)
df["Current Ver"].unique()

array(['1.0', '2.0', '1.2', '0', '1.1', '6.1', '2.9', '2.8', '3.8', '3.1',
       '2.2', '5.5', '4.0', '1.5', '6.0', '6.7', '1.6', '2.1', '1.3', '1',
       '1.4', '3.0', '1.7', '2.5', '2.4', '1.9', '0.2', '4.4', '2.3',
       '8.3', '3.2', '4.6', '5.4', '3.3', '9.1', '5.0', '9.0', '7.0',
       '3.9', '3.4', '9.7', '4.9', '5.2', '3.5', '7.3', '5.1', '4.1', '9',
       '0.9', '6.4', '3.7', '2.6', '4.3', '6.3', '4.7', '4.2', '1.8', '3',
       '9.8', '2', '4', '6', '7.5', '9.9', '0.1', '7.7', '7.2', '6.5',
       '5.6', '4.5', '5.8', '8.1', '6.6', '4.8', '8.7', '0.0', '3.6',
       '5.3', '8.2', '5.7', '8.5', '8.0', '8.4', '6.8', '6.2', '2.7',
       '5.9', '7.8', '7.1', '7.9', '8.8', '7.4', '9.6', '8', '9.4', '5',
       '0.7', '0.5', '7', '7.6', '0.6', '8.9', '6.9', '9.3', '9.2', '8.6',
       '9.5', '0.8', '0.4', '0.3'], dtype=object)

In [795]:
# seen as categorical and order it
df["Current Ver"] = pd.to_numeric(df["Current Ver"])
uniq_ver = df["Current Ver"].unique()
where_are_nan = np.isnan(uniq_ver)
uniq_ver[where_are_nan] = 0
ord_ver = sorted(uniq_ver)

In [796]:
df["Android Ver"].unique()

array(['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up',
       '3.0 and up', '4.1 and up', '4.0 and up', '2.3.3 and up',
       'Varies with device', '2.2 and up', '5.0 and up', '6.0 and up',
       '1.6 and up', '1.5 and up', '2.1 and up', '7.0 and up',
       '4.3 and up', '4.0.3 - 7.1.1', '2.0 and up', '3.2 and up',
       '4.4W and up', '5.1 and up', '7.1 and up', '7.0 - 7.1.1',
       '8.0 and up', '5.0 - 8.0', '3.1 and up', '2.0.1 and up',
       '4.1 - 7.1.1', '5.0 - 6.0', '1.0 and up'], dtype=object)

In [798]:
# split
df["Begin Ver"]=df["Android Ver"].apply(lambda x:str(x).split(' and ')[0].split(' - ')[0])
df["Begin Ver"]=df["Begin Ver"].replace('4.4W','4.4')
df["End Ver"]=df["Android Ver"].apply(lambda x:str(x).split(' and ')[-1].split(' - ')[-1])

In [799]:
a1 = df["Begin Ver"].unique()
ord_begin = sorted(a1)

In [800]:
df["End Ver"].unique()

array(['up', 'Varies with device', '7.1.1', '8.0', '6.0'], dtype=object)

In [801]:
a2 = df["End Ver"].unique()
ord_end = sorted(a2)
ord_end

['6.0', '7.1.1', '8.0', 'Varies with device', 'up']

In [802]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 10840
Data columns (total 18 columns):
App               9360 non-null object
Category          9360 non-null object
Rating            9360 non-null float64
Reviews           9360 non-null int64
Size              9360 non-null float64
Installs          9360 non-null object
Type              9360 non-null object
Price             9360 non-null float64
Content Rating    9360 non-null object
Genres            9360 non-null object
Last Updated      9360 non-null object
Current Ver       9360 non-null float64
Android Ver       9360 non-null object
First Genre       9360 non-null object
Second Genre      9360 non-null object
Updated Days      9360 non-null float64
Begin Ver         9360 non-null object
End Ver           9360 non-null object
dtypes: float64(5), int64(1), object(12)
memory usage: 1.4+ MB


In [803]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [804]:
df.drop(columns=['App'],inplace=True)
print(df.shape)
cat_ftrs = ['First Genre','Second Genre']
ordinal_ftrs = ['Installs','Type','Content Rating','Current Ver','Begin Ver','End Ver']
ordinal_cats = [['1+','5+','10+','50+','100+','500+','1,000+','5,000+','10,000+','50,000+','100,000+','500,000+',\
                 '1,000,000+','5,000,000+','10,000,000+','50,000,000+','100,000,000+','500,000,000+','1,000,000,000+'],\
                ['Free','Paid'],\
                ['Teen','Everyone 10+','Mature 17+','Adults only 18+','Everyone','Unrated'],\
                ord_ver,ord_begin,ord_end]
minmax_ftrs = ['Price']
standard_ftrs = ['Reviews','Size','Updated Days']
label_ftrs = ['Category']


(9360, 17)


In [805]:
# one-hot encoder
ohe = OneHotEncoder(sparse=False)
si = SimpleImputer(strategy='constant',fill_value='missing')
cat_values = ohe.fit_transform(si.fit_transform(df[cat_ftrs]))
cat_ftr_names = ohe.get_feature_names()
df_cat = pd.DataFrame(data=cat_values,columns = cat_ftr_names)

In [806]:
# ordinal encoder
oe = OrdinalEncoder(categories = ordinal_cats)
si2 = SimpleImputer(strategy='constant',fill_value='NA')
oe_values = oe.fit_transform(si2.fit_transform(df[ordinal_ftrs]))
df_ordinal = pd.DataFrame(data=oe_values,columns = ordinal_ftrs)

In [807]:
# standard scaler
ss = StandardScaler()
num_values = ss.fit_transform(df[standard_ftrs])
df_standard = pd.DataFrame(data=num_values,columns = standard_ftrs)

In [808]:
# minmax scaler
mms = MinMaxScaler()
minmax_values = ss.fit_transform(df[minmax_ftrs])
df_minmax = pd.DataFrame(data=minmax_values,columns = minmax_ftrs)

In [809]:
# label scaler
le = LabelEncoder()
le_values = le.fit_transform(df[label_ftrs])
df_le = pd.DataFrame(data=le_values)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [813]:
df_preprocessed = pd.concat([df_cat,df_ordinal,df_standard,df_minmax,df_le],axis=1)
df_preprocessed.shape

(9360, 111)